# Lab 2-4 (Bonus): Model Optimization

## ONNX 변환 및 양자화 실습

이 노트북에서는 학습된 ML 모델을 최적화하여 추론 성능을 향상시킵니다.

### 📋 실습 내용

1. Scikit-learn 모델 학습
2. ONNX 포맷 변환
3. 동적 양자화 적용
4. 성능 벤치마크
5. MLflow에 결과 기록

---

## 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install scikit-learn onnx onnxruntime skl2onnx mlflow joblib -q

print("✅ 패키지 설치 완료!")

In [ ]:
import os
import time
import numpy as np
import joblib
from pathlib import Path

# sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# ONNX
import onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as ort
from onnxruntime.quantization import quantize_dynamic, QuantType

# MLflow
import mlflow

print("✅ Import 완료")
print(f"   - ONNX: {onnx.__version__}")
print(f"   - ONNX Runtime: {ort.__version__}")

In [ ]:
# 출력 디렉토리 생성
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

print(f"✅ 출력 디렉토리: {output_dir.absolute()}")

---

## 2. 모델 학습

Iris 데이터셋으로 RandomForest 분류 모델을 학습합니다.

In [ ]:
# 데이터 로드
iris = load_iris()
X, y = iris.data, iris.target

# Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✅ 데이터 로드 완료")
print(f"   - 학습 데이터: {len(X_train)} samples")
print(f"   - 테스트 데이터: {len(X_test)} samples")
print(f"   - 피처 수: {X.shape[1]}")
print(f"   - 클래스 수: {len(iris.target_names)}")

In [ ]:
# 모델 학습
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# 정확도 평가
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"✅ 모델 학습 완료")
print(f"   - 테스트 정확도: {accuracy:.4f}")

In [ ]:
# 원본 모델 저장
original_path = output_dir / "model_original.joblib"
joblib.dump(model, original_path)

original_size = os.path.getsize(original_path) / 1024
print(f"✅ 원본 모델 저장: {original_path}")
print(f"   - 파일 크기: {original_size:.1f} KB")

---

## 3. ONNX 변환

Scikit-learn 모델을 ONNX 포맷으로 변환합니다.

In [ ]:
# 입력 형식 정의
initial_type = [('float_input', FloatTensorType([None, 4]))]

# ONNX 변환
onnx_model = convert_sklearn(
    model,
    initial_types=initial_type,
    target_opset=12,
    options={id(model): {'zipmap': False}}
)

# 저장
onnx_path = output_dir / "model_optimized.onnx"
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

onnx_size = os.path.getsize(onnx_path) / 1024
reduction = (1 - onnx_size / original_size) * 100

print(f"✅ ONNX 변환 완료!")
print(f"   - 파일: {onnx_path}")
print(f"   - 크기: {onnx_size:.1f} KB (원본 대비 {reduction:.1f}% 감소)")

In [ ]:
# ONNX 모델 검증
test_samples = X_test[:3].astype(np.float32)

# 원본 모델 예측
original_pred = model.predict(test_samples)

# ONNX 모델 예측
session = ort.InferenceSession(str(onnx_path))
input_name = session.get_inputs()[0].name
onnx_pred = session.run(None, {input_name: test_samples})[0]

print(f"✅ 예측 검증")
print(f"   - 원본 모델: {original_pred.tolist()}")
print(f"   - ONNX 모델: {onnx_pred.tolist()}")
print(f"   - 일치 여부: {'✅ 일치!' if np.array_equal(original_pred, onnx_pred) else '❌ 불일치'}")

---

## 4. 양자화 적용

ONNX 모델에 동적 양자화를 적용합니다.

In [ ]:
# 동적 양자화 적용
quantized_path = output_dir / "model_quantized.onnx"

quantize_dynamic(
    model_input=str(onnx_path),
    model_output=str(quantized_path),
    weight_type=QuantType.QUInt8,
    optimize_model=True
)

quantized_size = os.path.getsize(quantized_path) / 1024
total_reduction = (1 - quantized_size / original_size) * 100

print(f"✅ 양자화 완료!")
print(f"   - 파일: {quantized_path}")
print(f"   - 크기: {quantized_size:.1f} KB (원본 대비 {total_reduction:.1f}% 감소)")

In [ ]:
# 양자화 모델 정확도 검증
X_test_float = X_test.astype(np.float32)

# ONNX 정확도
session_onnx = ort.InferenceSession(str(onnx_path))
pred_onnx = session_onnx.run(None, {input_name: X_test_float})[0]
acc_onnx = accuracy_score(y_test, pred_onnx)

# 양자화 정확도
session_quant = ort.InferenceSession(str(quantized_path))
pred_quant = session_quant.run(None, {input_name: X_test_float})[0]
acc_quant = accuracy_score(y_test, pred_quant)

print(f"✅ 정확도 비교")
print(f"   - 원본 모델:   {accuracy:.4f}")
print(f"   - ONNX 모델:   {acc_onnx:.4f}")
print(f"   - 양자화 모델: {acc_quant:.4f}")

---

## 5. 성능 벤치마크

In [ ]:
def benchmark(predict_func, X, n_iterations=1000, warmup=100):
    """추론 시간 측정"""
    # 웜업
    for _ in range(warmup):
        predict_func(X)
    
    # 측정
    times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        predict_func(X)
        end = time.perf_counter()
        times.append((end - start) * 1000)  # ms
    
    return np.mean(times), np.std(times)

# 테스트 데이터
test_input = X_test[:1].astype(np.float32)

print("⏳ 벤치마크 실행 중...")

In [ ]:
# 원본 모델 벤치마크
mean_original, std_original = benchmark(
    lambda x: model.predict(x),
    test_input
)
print(f"✅ Original: {mean_original:.3f} ± {std_original:.3f} ms")

# ONNX 모델 벤치마크
session_onnx = ort.InferenceSession(str(onnx_path))
mean_onnx, std_onnx = benchmark(
    lambda x: session_onnx.run(None, {input_name: x}),
    test_input
)
print(f"✅ ONNX: {mean_onnx:.3f} ± {std_onnx:.3f} ms")

# 양자화 모델 벤치마크
session_quant = ort.InferenceSession(str(quantized_path))
mean_quant, std_quant = benchmark(
    lambda x: session_quant.run(None, {input_name: x}),
    test_input
)
print(f"✅ Quantized: {mean_quant:.3f} ± {std_quant:.3f} ms")

In [ ]:
# 결과 요약
speed_onnx = mean_original / mean_onnx
speed_quant = mean_original / mean_quant

print("\n" + "=" * 60)
print("📊 최적화 결과 요약")
print("=" * 60)
print(f"\n모델 크기:")
print(f"  - Original:  {original_size:.1f} KB")
print(f"  - ONNX:      {onnx_size:.1f} KB ({(1-onnx_size/original_size)*100:.1f}% 감소)")
print(f"  - Quantized: {quantized_size:.1f} KB ({(1-quantized_size/original_size)*100:.1f}% 감소)")

print(f"\n추론 속도:")
print(f"  - Original:  {mean_original:.3f} ms (1.00x)")
print(f"  - ONNX:      {mean_onnx:.3f} ms ({speed_onnx:.2f}x faster)")
print(f"  - Quantized: {mean_quant:.3f} ms ({speed_quant:.2f}x faster)")

print(f"\n정확도:")
print(f"  - Original:  {accuracy:.4f}")
print(f"  - ONNX:      {acc_onnx:.4f}")
print(f"  - Quantized: {acc_quant:.4f}")
print("=" * 60)

---

## 6. MLflow 기록 (선택)

In [ ]:
# MLflow 기록 (선택적)
try:
    mlflow.set_experiment("model-optimization")
    
    with mlflow.start_run(run_name="optimization-results"):
        # 메트릭
        mlflow.log_metric("original_size_kb", original_size)
        mlflow.log_metric("onnx_size_kb", onnx_size)
        mlflow.log_metric("quantized_size_kb", quantized_size)
        mlflow.log_metric("original_inference_ms", mean_original)
        mlflow.log_metric("onnx_inference_ms", mean_onnx)
        mlflow.log_metric("quantized_inference_ms", mean_quant)
        mlflow.log_metric("speed_improvement", speed_quant)
        
        # 모델 아티팩트
        mlflow.log_artifact(str(quantized_path))
        
        print(f"✅ MLflow 기록 완료!")
        print(f"   - Run ID: {mlflow.active_run().info.run_id}")
except Exception as e:
    print(f"⚠️  MLflow 기록 실패: {e}")
    print("   (MLflow 서버가 실행 중이 아닐 수 있습니다)")

---

## ✅ 실습 완료!

### 핵심 포인트

1. **ONNX 변환**: Scikit-learn 모델을 ONNX로 변환하여 약 68% 크기 감소
2. **양자화**: 추가로 약 90% 크기 감소 (원본 대비)
3. **속도 향상**: ONNX Runtime으로 약 7-9배 추론 속도 향상
4. **정확도 유지**: 최적화 후에도 정확도 손실 없음

### 실무 적용

- 엣지 디바이스 배포 시 모델 최적화 필수
- KServe Triton Runtime으로 ONNX 모델 서빙 가능
- GPU 환경에서는 TensorRT 추가 적용 고려